In [1]:
# System imports
from datetime import datetime
import numpy as np
import pandas as pd

# Third-party imports
import yfinance as yf
from pathlib import Path

# Local imports
from portfolio.holdings import Portfolio
from portfolio.transactions import Transactions
import datehandler

# Vars
today = datetime.today().date()

ModuleNotFoundError: No module named 'portfolio.commsec'

In [3]:
trades_df = Transactions().all

p_dates = datehandler.date_list(trades_df.index[-1], today)  # Portfolio dates index

tickers = list(sorted(set(trades_df.Ticker.to_list())))
props = ['TradeType','TradeVolume','TradePrice','HoldingVolume','Vwap']

columns = pd.MultiIndex.from_product([tickers,props], names=['Tickers','Props'])
df_p = pd.DataFrame(None, index=pd.DatetimeIndex(p_dates), columns=columns)  # Create multiindex portfolio df

# Clean index and columns
df_p.index.name = 'Date'
df_p = df_p.reindex(sorted(df_p.columns),axis=1)  # Sort columns
df_p = df_p.reindex(sorted(df_p.index),axis=0)  # Sort index
# df_p

NameError: name 'Trades' is not defined

In [4]:
dropped_cols = ['Market','Ticker','TradePrice','Brokerage']
for ticker in tickers:
    # Add trade dataframe values to portfolio dataframe
    df_t = trades_df[trades_df.Ticker == ticker].copy()
    df_t.loc[df_t['TradeType'] == 'S', 'Volume'] *= -1  # Don't worry about the warning
    df_t = df_t.drop(columns=dropped_cols)  # Drop columns first
    df_t = df_t.rename(columns={            # Then rename columns - due to conflicting name!
        'Volume':'TradeVolume',
        'EffectivePrice':'TradePrice',
        })
    df_p[ticker] = df_t

NameError: name 'tickers' is not defined

In [5]:
# Lookup prices
# Build list of tickers for yfinance
lookup_tickers = [f'{ticker}.AX' for ticker in df_p.columns.levels[0].to_list()]
lookup_tickers = ' '.join(lookup_tickers)

prices = yf.download(lookup_tickers, start=p_dates[0], end=p_dates[-1], group_by='ticker')

NameError: name 'df_p' is not defined

In [6]:
tickers_with_prices = []

# Remove market suffix from data
tickers_without_suffix = [ticker.replace('.AX','') for ticker in prices.columns.levels[0]]

prices.columns.set_levels(tickers_without_suffix, level=0, inplace=True)

NameError: name 'prices' is not defined

In [7]:
def _build_from_trades(df):
    df_ticker = df.copy()

    df_ticker['HoldingVolume'] = df_ticker['TradeVolume'].fillna(0).cumsum()

    # Vwap: BookValue / HoldingVolume
    df_ticker.loc[df_ticker['TradeType'] == 'B','TradeEncoding'] = 1
    df_ticker.loc[df_ticker['TradeType'] == 'S','TradeEncoding'] = 0
    df_ticker['BuyValue'] = df_ticker['TradeVolume'] * df_ticker['TradePrice'] * df_ticker['TradeEncoding']
    df_ticker['BuyVolume'] = df_ticker['TradeVolume'] * df_ticker['TradeEncoding']
    df_ticker['BuyVolume'] = df_ticker['BuyVolume'].cumsum()  # Temp calc column
    df_ticker['BuyValue'] = df_ticker['BuyValue'].cumsum()  # Temp calc column
    df_ticker['Vwap'] = df_ticker['BuyValue'].divide(df_ticker['BuyVolume']).fillna(method='ffill')

    df_ticker = df_ticker.drop(columns=['BuyVolume','BuyValue','TradeEncoding'])  # Remove temp calc columns
    
    return df_ticker

def _stocksplits(df):
    try:  # Dividends and stock splits
        stock = yf.Ticker(f'{ticker}.AX')
        df_actions = stock.actions
    except TypeError:
        print(f'\rSomething went wrong!',flush=True)
        return
    
    # Add stocksplits column
    if len(df_actions[df_actions['Stock Splits'] > 0]) > 0:
        df['StockSplits'] = df_actions['Stock Splits'].replace(0,np.nan)

        # Set all dates with 0 holdings to have np.nan holdings
        df['HoldingVolume'] = df['HoldingVolume'].replace(0,np.nan)

        stocksplit_cumulative = df.loc[df['HoldingVolume'].first_valid_index():]['StockSplits'].cumprod()
        stocksplit_cumulative = stocksplit_cumulative.fillna(method='ffill').fillna(1)


        # Update holding volume
        df['HoldingVolume'] *= stocksplit_cumulative
        df['HoldingVolume'] = np.ceil(df['HoldingVolume'])
        df['HoldingVolume'] = df['HoldingVolume'].fillna(0)  # Replace np.nan holdings with 0
        
        # Update vwap
        df['Vwap'] /= stocksplit_cumulative

    return df

# -------Code ---------
# Build portfolio
for count, ticker in enumerate(df_p.columns.levels[0]):
    print(f'\r{ticker} | Progress {count+1}/{len(df_p.columns.levels[0])} ',end='',flush=True)
    
    # ==== Test
    if ticker != 'RMD':  # For testing one ticker only
        continue
    # ==== Test

    if ticker not in tickers_without_suffix:
        print(f'Skipping {ticker} due to no price data...')
        continue
    
    df_t = df_p[ticker]

    df_t = _build_from_trades(df_t)     # Build porfolio from trades
    df_t['ClosePrice'] = prices[ticker]['Close'].copy()  # Load prices into dataframe
    df_t = _stocksplits(df_t)  # Update dataframe with stocksplits
    ####################################
    #### Dividends to be added manually  
    ####################################

    # Calculate cashflow
    df_t['Cashflow'] = df_t['TradePrice'] * df_t['TradeVolume']
    df_t['Cashflow'] = df_t['Cashflow'].fillna(0)
    
    # Calculate end value
    df_t['EndValue'] = df_t['HoldingVolume'] * df_t['ClosePrice'].fillna(method='ffill')
    # Any trades have the end-value reset
    df_t.loc[df_t.TradeVolume > 0,'EndValue'] = df_t.loc[df_t.TradeVolume > 0]['Cashflow']
    # Add trade value to previous days close

    # Calculate start value
    df_t['StartValue'] = df_t['EndValue'].shift(1).replace(0,np.nan)

    # Calculate daily returns
    df_t['DailyReturn'] = df_t['EndValue'] / (df_t['StartValue'] + df_t['Cashflow']) - 1
    df_t['DailyReturn'] = df_t['DailyReturn'].replace(-1,np.nan) + 1  # Catch sales
    df_t['CumulativeReturn'] = df_t['DailyReturn'].cumprod() - 1

    # Drop group and re-append new df
    df_p = df_p.drop(ticker, axis='columns', level=0)
    for col in df_t.columns:
        df_p[ticker,col] = df_t[col]

    df_p[ticker].to_csv('test.csv')

# df_p[('portfolio', 'EndValue')] = df_p.xs('EndValue', level='Props', axis=1).sum(axis=1, min_count=1)
# df_p[('portfolio', 'StartValue')] = df_p.xs('StartValue', level='Props', axis=1).sum(axis=1, min_count=1)
# df_p[('portfolio', 'Cashflow')] = df_p.xs('Cashflow', level='Props', axis=1).sum(axis=1, min_count=1)
# df_p[('portfolio','DailyReturn')] = df_p[('portfolio','EndValue')]/(df_p[('portfolio','StartValue')]+df_p[('portfolio','Cashflow')]) - 1
# df_p['portfolio', 'DailyReturn'] = df_p['portfolio', 'DailyReturn'].replace(-1,np.nan) + 1
# df_p[('portfolio','CumulativeReturn')] = df_p[('portfolio','DailyReturn')].cumprod()

# Clean columns
# df_p = df_p.drop(['EndValue','StartValue','Cashflow'], axis='columns',level=1)

a = df_p['RMD'].loc['2016-04-27':]

NameError: name 'df_p' is not defined

In [8]:
a = df_p['RMD'].loc[df_p['RMD'].TradeVolume.isna() == False]

NameError: name 'df_p' is not defined